### Bring:
* raw genotypes in [MAP/PED format](http://pngu.mgh.harvard.edu/~purcell/plink/data.shtml)
    individual pedigree information and genotype calls
    * \*.ped file individual pedigree information and genotype calls
    * \*.map file - variant information file
   
These files can be generated by the **`populations`** program within **`Stacks`** using the "--plink" flag


### Take away:
* Set of genotypes passing filtering criteria.

### Programs used:
* [PLINK v1.9b](http://pngu.mgh.harvard.edu/~purcell/plink2/index.html)
    
### Steps
1. Setup
2. Remove whole populations.
3. Filter based on missingness.
4. Test for Hardy-Weinberg equilibrium (HWE) and quantify minor allele frequency (MAF) within each population.
5. Apply filters based on HWE and MAF.
6. Keep only one SNP per locus.
7. Summary statistics

## Step 1
### Setup

#### Imports

In [1]:
import numpy as np 
import pandas as pd
import collections
import os.path

#### Edit *.map file
If you have used a reference-based approach (i.e. `pstacks`), the \*.plink.map file produced by `Stacks` may have too many different contig names for PLINK.  
When I run it out of the box, I get the following error message:

*"Error: Too many distinct nonstandard chromosome/contig names"*

\*.map files have one line per SNP and four columns, the first column give hte 'chr' or 'contig' location.

If you have used `ustacks`, the 'chr' column will already be filled with 0's and you should be good to go already.

#### Fix:
Replace these distinct contigs names with either all '0' or an integer representing a placement on a linkage group.

#### Replace the chr column with all 0's
This is not what I did, it is just an example.

In [2]:
##### not run, replace insert_map_file_name_here with your map filename
# link_map_file = {insert_map_file_name_here}
# with open(plink_map_file) as INFILE:
#    map_lines = INFILE.readlines()
#
# with open(plink_map_file+".bak", "w") as OUTFILE:
#    OUTFILE.writelines(map_lines)
#
# map_lines_iter = iter(map_lines)
# with open(plink_map_file, "w") as OUTFILE:
#     #write header
#     OUTFILE.write(next(map_lines_iter))
#     for xx in map_lines_iter:
#         yy = xx.split("\t")
#         OUTFILE.write("\t".join(['0', yy[1], yy[2], yy[3]]))

#### Replace the contig names given by Stacks with placements on a linkage map. 

Change contigs IDs to reflect placement on linkage groups, unplaced contigs are changed to "0". Notice the second column of the map file still contains information on the Stacks catalog ID and variant position.

Make sure you have a backup \*.plink.map file.

In [3]:
plink_map_file = os.path.join('data','batch_4','pop_genotypes','batch_4.plink.map')
with open(plink_map_file) as INFILE:
    map_lines = INFILE.readlines()
    
with open(plink_map_file+".bak", "w") as OUTFILE:
    OUTFILE.writelines(map_lines)

In [4]:
linkage_map = pd.read_csv(os.path.join('linkage_map','LEPmap','with_paralogs','final','PS_chum_map_2015.txt'), sep = "\t")
LG_of_SNP = dict(zip(linkage_map['stacks_SNP'], linkage_map['paper1_LG']))

map_lines_iter = iter(map_lines)
with open(plink_map_file, "w") as OUTFILE:
    #write header
    OUTFILE.write(next(map_lines_iter))
    for xx in map_lines_iter:
        contig, stacks_SNP, pos1, pos2 = xx.strip().split("\t")
        OUTFILE.write("{}\t{}\t{}\t{}\n".format(LG_of_SNP.get(stacks_SNP, 0), stacks_SNP, pos1, pos2))

#### setup plink

Change to the directory with your map/ped files.  The filtering process will generate many intermediate files, you may want to create a separate directory.

In [5]:
cd /home/ipseg/Desktop/waples/chum_populations/data/batch_4/pop_genotypes

/home/ipseg/Desktop/waples/chum_populations/data/batch_4/pop_genotypes


In [6]:
raw_genotypes  = "batch_4.plink"
univseral_plink_commands = "--allow-extra-chr --allow-no-sex --write-snplist --autosome-num 50 --keep-allele-order "

!pwd

/home/ipseg/Desktop/waples/chum_populations/data/batch_4/pop_genotypes


#### filtering parameters

In [7]:
pops_to_keep = ['1','2','3','4','5','6','7','8','9','10']

# missingness
locus_max_miss = 0.25
ind_max_miss = 0.25

# MAF
local_MAF_threshold = 0.05
global_MAF_threshold = 0.05

# HWE
local_HWE_pval = .05
HWE_pop_threshold = 5

## Step 2
### Remove collections / populations

If you have samples from different populations and want to exclude any whole populations, now is the time.

In [8]:
!plink --file {raw_genotypes} {univseral_plink_commands} \
    --family --keep-cluster-names {" ".join(pops_to_keep)} --make-bed --out filter_pops

PLINK v1.90b3q 64-bit (29 May 2015)        https://www.cog-genomics.org/plink2
(C) 2005-2015 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to filter_pops.log.
Options in effect:
  --allow-extra-chr
  --allow-no-sex
  --autosome-num 50
  --family
  --file batch_4.plink
  --keep-allele-order
  --keep-cluster-names 1 2 3 4 5 6 7 8 9 10
  --make-bed
  --out filter_pops
  --write-snplist

32127 MB RAM detected; reserving 16063 MB for main workspace.
.ped scan complete (for binary autoconversion).
Performing single-pass .bed write (28861 variants, 200 samples).
--file: filter_pops-temporary.bed + filter_pops-temporary.bim +
filter_pops-temporary.fam written.
28861 variants loaded from .bim file.
200 samples (0 males, 0 females, 200 ambiguous) loaded from .fam.
Ambiguous sex IDs written to filter_pops.nosex .
--family: 10 clusters defined.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 200 founders and 0 nonfounders present.
Cal

## Step 3
### Filter based on missingness

#### Remove samples not genotyed at least X% of loci	 

In [9]:
!plink --bfile filter_pops --mind {ind_max_miss} {univseral_plink_commands} \
    --make-bed --out filter_inds

PLINK v1.90b3q 64-bit (29 May 2015)        https://www.cog-genomics.org/plink2
(C) 2005-2015 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to filter_inds.log.
Options in effect:
  --allow-extra-chr
  --allow-no-sex
  --autosome-num 50
  --bfile filter_pops
  --keep-allele-order
  --make-bed
  --mind 0.25
  --out filter_inds
  --write-snplist

32127 MB RAM detected; reserving 16063 MB for main workspace.
28861 variants loaded from .bim file.
200 samples (0 males, 0 females, 200 ambiguous) loaded from .fam.
Ambiguous sex IDs written to filter_inds.nosex .
26 samples removed due to missing genotype data (--mind).
IDs written to filter_inds.irem .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 174 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30

#### Remove loci not genotyed at least X% of samples

In [10]:
!plink --bfile  filter_inds --geno {locus_max_miss} {univseral_plink_commands} \
    --make-bed --out miss_final

PLINK v1.90b3q 64-bit (29 May 2015)        https://www.cog-genomics.org/plink2
(C) 2005-2015 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to miss_final.log.
Options in effect:
  --allow-extra-chr
  --allow-no-sex
  --autosome-num 50
  --bfile filter_inds
  --geno 0.25
  --keep-allele-order
  --make-bed
  --out miss_final
  --write-snplist

32127 MB RAM detected; reserving 16063 MB for main workspace.
28861 variants loaded from .bim file.
174 samples (0 males, 0 females, 174 ambiguous) loaded from .fam.
Ambiguous sex IDs written to miss_final.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 174 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42%43%44%45%4

## Step 4
### HWE and MAF within each population.

#### Create a separate file for each population.

In [11]:
for xx in pops_to_keep:
    !plink --bfile  miss_final --family --keep-cluster-names {xx} {univseral_plink_commands} \
    --make-bed --out fam_{xx}

PLINK v1.90b3q 64-bit (29 May 2015)        https://www.cog-genomics.org/plink2
(C) 2005-2015 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to fam_1.log.
Options in effect:
  --allow-extra-chr
  --allow-no-sex
  --autosome-num 50
  --bfile miss_final
  --family
  --keep-allele-order
  --keep-cluster-names 1
  --make-bed
  --out fam_1
  --write-snplist

32127 MB RAM detected; reserving 16063 MB for main workspace.
28800 variants loaded from .bim file.
174 samples (0 males, 0 females, 174 ambiguous) loaded from .fam.
Ambiguous sex IDs written to fam_1.nosex .
--family: 1 cluster defined.
154 samples removed by cluster filter(s).
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 20 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%

#### Calculate HWE statistics, write a list of snps passing HWE in each pop

In [12]:
for xx in pops_to_keep:
    !plink --bfile  fam_{xx} --hwe {local_HWE_pval} midp --hardy midp {univseral_plink_commands} \
    --out fam_{xx}.hwe

PLINK v1.90b3q 64-bit (29 May 2015)        https://www.cog-genomics.org/plink2
(C) 2005-2015 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to fam_1.hwe.log.
Options in effect:
  --allow-extra-chr
  --allow-no-sex
  --autosome-num 50
  --bfile fam_1
  --hardy midp
  --hwe 0.05 midp
  --keep-allele-order
  --out fam_1.hwe
  --write-snplist

32127 MB RAM detected; reserving 16063 MB for main workspace.
28800 variants loaded from .bim file.
20 samples (0 males, 0 females, 20 ambiguous) loaded from .fam.
Ambiguous sex IDs written to fam_1.hwe.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 20 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42%43%44%45%46%4

#### calculate MAF, make a list of snps with MAF > threshold in each pop

In [13]:
for xx in pops_to_keep:
    !plink --bfile  fam_{xx} --maf {local_MAF_threshold} {univseral_plink_commands} \
    --out fam_{xx}.maf
    
# Alternatively, you can use minor allele counts (not yet implemetned in PLINK1.9b)
#for xx in pops_to_keep:
#    !plink --bfile  fam_{xx} --mac 3 {univseral_plink_commands} --write-snplist --out fam_{xx}.maf

PLINK v1.90b3q 64-bit (29 May 2015)        https://www.cog-genomics.org/plink2
(C) 2005-2015 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to fam_1.maf.log.
Options in effect:
  --allow-extra-chr
  --allow-no-sex
  --autosome-num 50
  --bfile fam_1
  --keep-allele-order
  --maf 0.05
  --out fam_1.maf
  --write-snplist

32127 MB RAM detected; reserving 16063 MB for main workspace.
28800 variants loaded from .bim file.
20 samples (0 males, 0 females, 20 ambiguous) loaded from .fam.
Ambiguous sex IDs written to fam_1.maf.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 20 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42%43%44%45%46%47%48%49%50%

## Step 5
### Apply filters

In [14]:
passing_MAF = collections.defaultdict(int)
for xx in pops_to_keep:
    with open("fam_{}.maf.snplist".format(xx)) as INFILE:
        for line in INFILE:
            passing_MAF[line.strip()] +=1

with open("passing_MAF.snplist", 'w') as OUTFILE:
    for locus in passing_MAF.keys():
        OUTFILE.write(locus + "\n")

passing_HWE = collections.defaultdict(int)
for xx in pops_to_keep:
    with open("fam_{}.hwe.snplist".format(xx)) as INFILE:
        for line in INFILE:
            passing_HWE[line.strip()] +=1

with open("passing_HWE.snplist", 'w') as OUTFILE:
    for locus, pass_cnt in passing_HWE.items():
        if pass_cnt >= HWE_pop_threshold:
            OUTFILE.write(locus + "\n")

In [15]:
# remove loci failing HWE or MAF filters locally
!plink --bfile  miss_final --extract passing_MAF.snplist {univseral_plink_commands} \
    --make-bed --out filter_MAF
!plink --bfile  filter_MAF --extract passing_HWE.snplist {univseral_plink_commands} \
    --make-bed --out filter_HWE

PLINK v1.90b3q 64-bit (29 May 2015)        https://www.cog-genomics.org/plink2
(C) 2005-2015 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to filter_MAF.log.
Options in effect:
  --allow-extra-chr
  --allow-no-sex
  --autosome-num 50
  --bfile miss_final
  --extract passing_MAF.snplist
  --keep-allele-order
  --make-bed
  --out filter_MAF
  --write-snplist

32127 MB RAM detected; reserving 16063 MB for main workspace.
28800 variants loaded from .bim file.
174 samples (0 males, 0 females, 174 ambiguous) loaded from .fam.
Ambiguous sex IDs written to filter_MAF.nosex .
--extract: 28800 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 174 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%3

#### global MAF filter

In [16]:
!plink --bfile  filter_HWE --maf {global_MAF_threshold} {univseral_plink_commands} --freq \
    --make-bed --out filter_HWE_MAF

PLINK v1.90b3q 64-bit (29 May 2015)        https://www.cog-genomics.org/plink2
(C) 2005-2015 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to filter_HWE_MAF.log.
Options in effect:
  --allow-extra-chr
  --allow-no-sex
  --autosome-num 50
  --bfile filter_HWE
  --freq
  --keep-allele-order
  --maf 0.05
  --make-bed
  --out filter_HWE_MAF
  --write-snplist

32127 MB RAM detected; reserving 16063 MB for main workspace.
20480 variants loaded from .bim file.
174 samples (0 males, 0 females, 174 ambiguous) loaded from .fam.
Ambiguous sex IDs written to filter_HWE_MAF.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 174 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42%

## Step 6 
### Retain a single SNP at each locus
#### select based on maximum MAF

In [17]:
maf_frq = pd.read_csv("filter_HWE_MAF.frq", sep=" ", skipinitialspace = True)

maf_frq['catID']  =  [int(xx[0]) for xx in maf_frq['SNP'].str.split("_", n = 1)]
maf_frq['pos']  = [int(xx[1]) for xx in maf_frq['SNP'].str.split("_", n = 1)]
maf_frq.sort(['catID', 'MAF'], ascending = False, inplace = True)
single_catID = maf_frq.drop_duplicates(subset = ['catID'])
single_catID['SNP'].to_csv('single_catID.snplist', index = False)

In [18]:
!plink --bfile  filter_HWE_MAF --extract single_catID.snplist \
    {univseral_plink_commands} --make-bed --recode 12 A tabx --out complete

PLINK v1.90b3q 64-bit (29 May 2015)        https://www.cog-genomics.org/plink2
(C) 2005-2015 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to complete.log.
Options in effect:
  --allow-extra-chr
  --allow-no-sex
  --autosome-num 50
  --bfile filter_HWE_MAF
  --extract single_catID.snplist
  --keep-allele-order
  --make-bed
  --out complete
  --recode 12 A tabx
  --write-snplist

32127 MB RAM detected; reserving 16063 MB for main workspace.
20288 variants loaded from .bim file.
174 samples (0 males, 0 females, 174 ambiguous) loaded from .fam.
Ambiguous sex IDs written to complete.nosex .
--extract: 13407 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 174 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%

### Data subsets

In [19]:
## create a file listing a list of stacks_SNP IDs that are on contigs marked as paralogs
linkage_map = pd.read_csv(os.path.join('/home','ipseg','Desktop','waples','chum_populations','linkage_map','LEPmap','with_paralogs','final','PS_chum_map_2015.txt'), sep = "\t")
contig_of_SNP = dict(zip(linkage_map['stacks_SNP'], linkage_map['contig']))
paralog_contigs = pd.read_csv('/home/ipseg/Desktop/waples/chum_populations/linkage_map/chum_paralogs.txt', header = None)
paralog_contigs.columns = [['paralogs']]
paralog_contigs['paralogs'] = ['c'+str(xx) for xx in paralog_contigs['paralogs'] ]
SNPs_on_paralogs = [xx for xx in linkage_map['stacks_SNP'] if (contig_of_SNP[xx] in list(paralog_contigs['paralogs'])) ]

with open('/home/ipseg/Desktop/waples/chum_populations/data/batch_4/pop_genotypes/paralog_SNPs.txt', 'w') as OUTFILE:
    OUTFILE.write('\n'.join(SNPs_on_paralogs))

In [20]:
# complete
!plink --bfile complete {univseral_plink_commands} --recode -make-bed --out complete
# on linkage map
!plink --bfile complete --not-chr 0 {univseral_plink_commands} --recode --make-bed --out on_map
# non paralogs
!plink --bfile complete --exclude paralog_SNPs.txt {univseral_plink_commands} \
    --recode --make-bed --out non_paralogs
# mapped_non paralogs
!plink --bfile complete --not-chr 0 --exclude paralog_SNPs.txt {univseral_plink_commands} \
    --recode --make-bed --out mapped_non_paralogs

PLINK v1.90b3q 64-bit (29 May 2015)        https://www.cog-genomics.org/plink2
(C) 2005-2015 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to complete.log.
Options in effect:
  --allow-extra-chr
  --allow-no-sex
  --autosome-num 50
  --bfile complete
  --keep-allele-order
  --make-bed
  --out complete
  --recode
  --write-snplist

32127 MB RAM detected; reserving 16063 MB for main workspace.
Note: --make-bed input and output filenames match.  Appending '~' to input
filenames.
13407 variants loaded from .bim file.
174 samples (0 males, 0 females, 174 ambiguous) loaded from .fam.
Ambiguous sex IDs written to complete.nosex .
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 174 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%

## Summary Statistics 

Here I will use PLINK to calculate some summary statistics from our freshly filtered population data.  These results may inform our filtering decisions. See plink documentation for the details of the statistics and file formats.

In [21]:
def summarize_plink(input_name):
    !plink --bfile {input_name} --family --missing --freq --het small-sample --ibc --fst \
        {univseral_plink_commands} --out {input_name}
    !plink --bfile {input_name} --family --freqx {univseral_plink_commands} --out {input_name}
    !plink --bfile {input_name} {univseral_plink_commands} --recode A --out {input_name}

In [22]:
summarize_plink('complete')
summarize_plink('on_map')
summarize_plink('non_paralogs')
summarize_plink('mapped_non_paralogs')

PLINK v1.90b3q 64-bit (29 May 2015)        https://www.cog-genomics.org/plink2
(C) 2005-2015 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to complete.log.
Options in effect:
  --allow-extra-chr
  --allow-no-sex
  --autosome-num 50
  --bfile complete
  --family
  --freq
  --fst
  --het small-sample
  --ibc
  --keep-allele-order
  --missing
  --out complete
  --write-snplist

32127 MB RAM detected; reserving 16063 MB for main workspace.
13407 variants loaded from .bim file.
174 samples (0 males, 0 females, 174 ambiguous) loaded from .fam.
Ambiguous sex IDs written to complete.nosex .
--family: 10 clusters defined.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 174 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%

In [23]:
#!plink --bfile on_map --family --missing {univseral_plink_commands} --out on_map
#!plink --bfile on_map --family --freq {univseral_plink_commands} --out on_map
#!plink --bfile on_map --family --het small-sample --ibc {univseral_plink_commands} --out on_map
#!plink --bfile on_map --family --freqx --fst {univseral_plink_commands} --out on_map

In [24]:
#!plink --bfile non_paralogs --family --missing {univseral_plink_commands} --out non_paralogs
#!plink --bfile non_paralogs --family --freq {univseral_plink_commands} --out non_paralogs
#!plink --bfile non_paralogs --family --het small-sample --ibc {univseral_plink_commands} --out non_paralogs
#!plink --bfile non_paralogs --family --freqx --fst {univseral_plink_commands} --out non_paralogs